In [1]:
# example of semi-supervised gan for mnist
import numpy as np
import os
import time

from numpy.random import randn, randint
import matplotlib.pyplot as plt

In [2]:
from keras.datasets.cifar10 import load_data
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input, Dense, Reshape, Flatten, Conv2D, Conv2DTranspose
from keras.layers import LeakyReLU, Dropout, Lambda, Activation, ReLU
from keras.layers import GlobalAveragePooling2D, BatchNormalization

from keras import backend as K

Using TensorFlow backend.


In [3]:
#!pip install -q -U tensorflow-addons==0.11.2

In [4]:
#from tensorflow_addons.layers import WeightNormalization

In [5]:
from utils import verifyDir
from utils.networks import normalize, unnormalize, plot_data

In [6]:
# os.environ["CUDA_VISIBLE_DEVICES"]="1"

### Parameters

In [7]:
x_height, x_width = [32, 32]
num_channels = 3
input_shape = (x_height, x_width, num_channels)
num_classes = 10

# learning rate
learning_rate = 0.0005
# optimizer
optimizer_grad = Adam(lr=learning_rate, beta_1=0.5)
# size of the latent space
latent_dim = 100

epochs=200
batch_size=128
labeled_rate = 1.0
labeled_samples = 50000*labeled_rate

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [8]:
LOG_PATH = f"Logs/SSGAN_CIFAR10/Classifier_{int(labeled_samples)}/"

In [9]:
verifyDir(LOG_PATH)

### Dataset

In [10]:
# load the images CIFAR10
def load_real_samples():
    # load dataset
    (trainX, trainy), (testX, testy) = load_data()
    # expand to 3d, e.g. add channels
    #X = np.expand_dims(trainX, axis=-1)
    #X_test = np.expand_dims(testX, axis=-1)
    testy = testy[:, 0]
    trainy = trainy[:,0]
    X_test = normalize(testX)
    X_train = normalize(trainX)
    print(X_train.shape, trainy.shape, X_test.shape, testy.shape)
    return [X_train, trainy], [X_test, testy]

### Discriminator

Custom Activation for the Unsupervised model

In [11]:
# custom activation function
def custom_activation(output):
    logexpsum = K.sum(K.exp(output), axis=-1, keepdims=True)
    result = logexpsum / (logexpsum + 1.0)
    return result

Defines a unsupersived output and supervised output.  
Both shared the same weights until the last Dense.

In [12]:
# define the standalone supervised and unsupervised discriminator models
def CIFAR10_discriminator(in_shape=(28,28,1), n_classes=10):
    # image input
    in_image = Input(shape=in_shape)
    # downsample 32x32 -> 16x16
    fe = Conv2D(64, (3, 3), strides=None, padding='same')(in_image)
    fe = Conv2D(128, (3,3), strides=(1,1), padding='same')(fe)
    fe = BatchNormalization()(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    
    fe = Conv2D(128, (3,3), strides=(2,2), padding='same')(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    
    fe = Conv2D(128, (3,3), strides=(2,2), padding='same')(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    # downsample 16x16 -> 8x8
    fe = Conv2D(256, (3,3), strides=(2,2), padding='same')(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    
    # Global Pooling feature maps
    fe = Flatten()(fe)
    # output layer nodes
    fe = Dense(n_classes)(fe)
    
    # supervised output
    c_out_layer = Activation('softmax')(fe)
    # define and compile supervised discriminator model
    supervised_model = Model(in_image, c_out_layer)
    supervised_model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer_grad, metrics=['accuracy'])
    
    # unsupervised output
    d_out_layer = Lambda(custom_activation)(fe)
    # define and compile unsupervised discriminator model
    unsupervised_model = Model(in_image, d_out_layer)
    unsupervised_model.compile(loss='binary_crossentropy', optimizer=optimizer_grad)
    
    return unsupervised_model, supervised_model

In [ ]:
un, su = CIFAR10_discriminator()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Adrian\anaconda3\envs\tf15\lib\site-packages\IPython\core\interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-13-113143402785>", line 1, in <module>
    un. su = CIFAR10_discriminator()
NameError: name 'un' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Adrian\anaconda3\envs\tf15\lib\site-packages\IPython\core\interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Adrian\anaconda3\envs\tf15\lib\site-packages\IPython\core\ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "C:\Users\Adr

In [ ]:
su.summary()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Adrian\anaconda3\envs\tf15\lib\site-packages\IPython\core\interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-13-523daf2c00e8>", line 1, in <module>
    su.summary()
NameError: name 'su' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Adrian\anaconda3\envs\tf15\lib\site-packages\IPython\core\interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Adrian\anaconda3\envs\tf15\lib\site-packages\IPython\core\ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "C:\Users\Adrian\anaconda3\envs\t

In [ ]:
os.exit()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Adrian\anaconda3\envs\tf15\lib\site-packages\IPython\core\interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-13-1191f1436809>", line 1, in <module>
    os.exit()
AttributeError: module 'os' has no attribute 'exit'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Adrian\anaconda3\envs\tf15\lib\site-packages\IPython\core\interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'AttributeError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Adrian\anaconda3\envs\tf15\lib\site-packages\IPython\core\ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "C:\Users\Adria

In [ ]:
# Discriminator Model Class Definition
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            # Block 1: input is (3) x 64 x 64
            nn.Conv2d(3, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # Block 2: input is (64) x 32 x 32
            nn.Conv2d(64, 64 * 2, 4, 2, 1, bias=False), # n_channels, out_channels, kernel_size, stride=1, padding=0,
            nn.BatchNorm2d(64 * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # Block 3: input is (64*2) x 16 x 16
            nn.Conv2d(64 * 2, 64 * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # Block 4: input is (64*4) x 8 x 8
            nn.Conv2d(64 * 4, 64 * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # Block 5: input is (64*8) x 4 x 4
            nn.Conv2d(64 * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid(),
            nn.Flatten()
            # Output: 1
        )

    def forward(self, input):
        out = self.main(input)
        return out.view(-1, 1).squeeze(1)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Adrian\anaconda3\envs\tf15\lib\site-packages\IPython\core\interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-13-4955810dfca6>", line 2, in <module>
    class Discriminator(nn.Module):
NameError: name 'nn' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Adrian\anaconda3\envs\tf15\lib\site-packages\IPython\core\interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Adrian\anaconda3\envs\tf15\lib\site-packages\IPython\core\ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "C:\Users\Adri

In [13]:
# define the standalone supervised and unsupervised discriminator models
def define_discriminator(in_shape=(32,32,3), n_classes=10):
    # image input
    in_image = Input(shape=in_shape)
    # downsample 32x32 -> 16x16
    fe = Dropout(0.2)(in_image)
    fe = WeightNormalization(Conv2D(96, (3,3), strides=(1,1), padding='same'), data_init=False)(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    fe = WeightNormalization(Conv2D(96, (3,3), strides=(1,1), padding='same'), data_init=False)(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    fe = WeightNormalization(Conv2D(96, (3,3), strides=(2,2), padding='same'), data_init=False)(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    fe = Dropout(0.5)(fe)
    
    # downsample 16x16 -> 8x8
    fe = WeightNormalization(Conv2D(192, (3,3), strides=(1,1), padding='same'), data_init=False)(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    fe = WeightNormalization(Conv2D(192, (3,3), strides=(1,1), padding='same'), data_init=False)(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    fe = WeightNormalization(Conv2D(192, (3,3), strides=(2,2), padding='same'), data_init=False)(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    fe = Dropout(0.5)(fe)
    
    # downsample 8x8 -> 4x4
    fe = WeightNormalization(Conv2D(192, (3,3), strides=(1,1), padding='valid'), data_init=False)(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    fe = WeightNormalization(Conv2D(192, (1,1), strides=(1,1), padding='valid'), data_init=False)(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    fe = WeightNormalization(Conv2D(192, (1,1), strides=(1,1), padding='valid'), data_init=False)(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    
    # Global Pooling feature maps
    fe = GlobalAveragePooling2D()(fe)
    # output layer nodes
    fe = WeightNormalization(Dense(n_classes))(fe)
    
    # supervised output
    c_out_layer = Activation('softmax')(fe)
    # define and compile supervised discriminator model
    supervised_model = Model(in_image, c_out_layer)
    supervised_model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer_grad, metrics=['accuracy'])
    
    # unsupervised output
    d_out_layer = Lambda(custom_activation)(fe)
    # define and compile unsupervised discriminator model
    unsupervised_model = Model(in_image, d_out_layer)
    unsupervised_model.compile(loss='binary_crossentropy', optimizer=optimizer_grad)
    
    return unsupervised_model, supervised_model

### Generator

In [ ]:
class Generator(nn.Module):
    def __init__(self, latent_vector_size = 100):
        super(Generator, self).__init__()
        
        self.deconv1 = nn.ConvTranspose2d( latent_vector_size, 64 * 8, 4, 1, 0, bias=False)
        self.norm1 = nn.BatchNorm2d(64 * 8)
        self.relu = nn.ReLU(True)
        self.deconv2 = nn.ConvTranspose2d(64 * 8, 64 * 4, 4, 2, 1, bias=False)
        self.norm2 = nn.BatchNorm2d(64 * 4)
        self.deconv3 = nn.ConvTranspose2d( 64 * 4, 64 * 2, 4, 2, 1, bias=False)
        self.norm3 = nn.BatchNorm2d(64*2)
        self.deconv4 = nn.ConvTranspose2d( 64*2 , 64, 4, 2, 1, bias=False)
        self.norm4 = nn.BatchNorm2d(64)
        self.deconv5 = nn.ConvTranspose2d( 64 , 3, 4, 2, 1, bias=False)
        self.tanh = nn.Tanh()
        
    
    def decode(self, z):
        z = self.relu(self.norm1(self.deconv1(z))) # b, 16, 5, 5
        z = self.relu(self.norm2(self.deconv2(z))) # b, 8, 15, 15
        z = self.relu(self.norm3(self.deconv3(z))) # b, 1, 28, 28
        z = self.relu(self.norm4(self.deconv4(z))) # b, 1, 28, 28
        z = self.tanh(self.deconv5(z))
        return z

    def forward(self, z):
        return self.decode(z)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



ERROR! Session/line number was not unique in database. History logging moved to new session 4630
Traceback (most recent call last):
  File "C:\Users\Adrian\anaconda3\envs\tf15\lib\site-packages\IPython\core\interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-14-d58cbd30cd88>", line 1, in <module>
    class Generator(nn.Module):
NameError: name 'nn' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Adrian\anaconda3\envs\tf15\lib\site-packages\IPython\core\interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Adrian\anaconda3\envs\tf15\lib\site-packages\IPython\core\ultratb.py", line 1101, in get_records
    r

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Adrian\anaconda3\envs\tf15\lib\site-packages\IPython\core\interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-14-d58cbd30cd88>", line 1, in <module>
    class Generator(nn.Module):
NameError: name 'nn' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Adrian\anaconda3\envs\tf15\lib\site-packages\IPython\core\interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Adrian\anaconda3\envs\tf15\lib\site-packages\IPython\core\interactiveshell.py", line 3361, in run_ast_nodes
    if (await self.run_code(code, result,  async_=asy)):
  File "C:\Users\Adrian\anaconda3\

In [14]:
# define the standalone generator model
def define_generator(latent_dim):
    # image generator input
    in_lat = Input(shape=(latent_dim,))
    # foundation for 4x4 image
    n_nodes = 512 * 4 * 4
    gen = Dense(n_nodes)(in_lat)
    gen = BatchNormalization()(gen)
    gen = ReLU()(gen)
    gen = Reshape((4, 4, 512))(gen)
    
    # upsample to 4x4 -> 8x8
    gen = Conv2DTranspose(256, (4,4), strides=(2,2), padding='same')(gen)
    gen = BatchNormalization()(gen)
    gen = ReLU()(gen)
    
    # upsample to 8x8 -> 16x16
    gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(gen)
    gen = BatchNormalization()(gen)
    gen = ReLU()(gen)
    
    # upsample to 16x16 -> 32x32
    #gen = Conv2DTranspose(3, (4,4), strides=(2,2), padding='same')(gen)
    gen = WeightNormalization(Conv2DTranspose(3, (4,4), strides=(2,2), padding='same'), data_init=False)(gen)
    
    # outlayer
    out_layer = Activation('tanh')(gen)
    
    # define model
    generator_model = Model(in_lat, out_layer)
    return generator_model

In [15]:
# define the standalone generator model
def CIFAR10_generator(latent_dim):
    # image generator input
    in_lat = Input(shape=(latent_dim,))
    # foundation for 4x4 image
    n_nodes = 256 * 4 * 4
    gen = Dense(n_nodes)(in_lat)
    gen = LeakyReLU(alpha=0.2)(gen)
    gen = Reshape((4, 4, 256))(gen)
    # upsample to 4x4 -> 8x8
    gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(gen)
    gen = LeakyReLU(alpha=0.2)(gen)
    
    # upsample to 8x8 -> 16x16
    gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(gen)
    gen = LeakyReLU(alpha=0.2)(gen)
    
    # upsample to 16x16 -> 32x32
    gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(gen)
    gen = LeakyReLU(alpha=0.2)(gen)
    
    # outlayer
    out_layer = Conv2D(3, (3,3), activation='tanh', padding='same')(gen)
    
    # define model
    generator_model = Model(in_lat, out_layer)
    return generator_model

ERROR! Session/line number was not unique in database. History logging moved to new session 4631


### Semi-Supervised GAN

We let the unsupervised model as not trainable, because we gonna trin the weights of the supervised model (which shares weights with the unsupervised).

In [16]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(generator_model, unsupervised_model):
    # make weights in the discriminator not trainable
    unsupervised_model.trainable = False
    # connect image output from generator as input to discriminator
    gan_output = unsupervised_model(generator_model.output)
    # define gan model as taking noise and outputting a classification
    model = Model(generator_model.input, gan_output)
    # compile model
    model.compile(loss='binary_crossentropy', optimizer=optimizer_grad)
    return model

### Selecting sub-set 

In [17]:
# select a supervised subset of the dataset, ensures classes are balanced
def select_supervised_samples(dataset, percent_samples=1.0, n_classes=10):
    X, y = dataset
    X_list, y_list = list(), list()
    #n_per_class = int(n_samples / n_classes)
    for i in range(n_classes):
        # get all images for this class
        X_with_class = X[y == i]
        # choose random instances
        ix = randint(0, len(X_with_class), int(len(X_with_class)*percent_samples))
        # add to list
        [X_list.append(X_with_class[j]) for j in ix]
        [y_list.append(i) for j in ix]
    return np.asarray(X_list), np.asarray(y_list)

### Getting Real Samples

In [18]:
# select real samples
def generate_real_samples(dataset, n_samples=100):
    # split into images and labels
    images, labels = dataset
    # choose random instances
    ix = randint(0, images.shape[0], n_samples)
    # select images and labels
    X, labels = images[ix], labels[ix]
    # generate class labels
    y = np.ones((n_samples, 1))
    return [X, labels], y

### Getting Fake Samples

In [19]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples=100):
    # generate points in the latent space
    z_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    z_input = z_input.reshape(n_samples, latent_dim)
    return z_input

In [20]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples=100):
    # generate points in latent space
    z_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    images = generator.predict(z_input)
    # create class labels
    y = np.zeros((n_samples, 1))
    return images, y

### Training

In [21]:
# train the generator and discriminator
def train(generator_model, unsupervised_model, supervised_model, gan_model, dataset, dataset_test, latent_dim, n_epochs=20, n_batch=100, percent_samples=1.0):
    # select supervised dataset
    X_sup, y_sup = select_supervised_samples(dataset, percent_samples=percent_samples)
    print(X_sup.shape, y_sup.shape)
    # calculate the number of batches per training epoch
    bat_per_epo = int(dataset[0].shape[0] / n_batch)
    # calculate the number of training iterations
    n_steps = bat_per_epo * n_epochs
    # calculate the size of half a batch of samples
    half_batch = int(n_batch / 2)
    print('n_epochs=%d, n_batch=%d, 1/2=%d, b/e=%d, steps=%d' % (n_epochs, n_batch, half_batch, bat_per_epo, n_steps))
    # manually enumerate epochs
    f_history = open(f"{LOG_PATH}SSL_GAN.csv", "w")
    f_history.write("step,generator_loss,unsupervised_real_loss,unsupervised_fake_loss,supervised_loss,supervised_acc,train_loss,test_loss,train_acc,test_acc\n")
    
    #for epoch in n_epochs:
    #    for batch in range(bat_per_epo):
    for step in range(1,n_steps+1):
#         t_start = time.time()
        # update supervised discriminator (c)
        [Xsup_real, ysup_real], _ = generate_real_samples([X_sup, y_sup], half_batch)
        c_loss, c_acc = supervised_model.train_on_batch(Xsup_real, ysup_real)
        
        # update unsupervised discriminator (d)
        [X_real, _], y_real = generate_real_samples(dataset, half_batch)
        d_loss1 = unsupervised_model.train_on_batch(X_real, y_real)
        
        X_fake, y_fake = generate_fake_samples(generator_model, latent_dim, half_batch)
        d_loss2 = unsupervised_model.train_on_batch(X_fake, y_fake)
        
        # update generator (g)
        X_gan, y_gan = generate_latent_points(latent_dim, n_batch), np.ones((n_batch, 1))
        g_loss = gan_model.train_on_batch(X_gan, y_gan)
#         t_total = (time.time() - t_start)
        # summarize loss on this batch
    
        # Train - Test
        X_train, y_train = dataset
        loss_train, acc_train = supervised_model.evaluate(X_train, y_train, verbose=0)

        # evaluate the test classifier model
        X_test, y_test = dataset_test
        loss_test, acc_test = supervised_model.evaluate(X_test, y_test, verbose=0)
        
        # Log
        print('step: %d | Train: G_Loss: %.3f, ' \
              'D_unsup_loss_real: %.3f, D_unsup_loss_fake: %.3f, ' \
              'D_sup_loss: %.3f, D_sup_acc: %.2f ' \
              'Train acc: %.3f Test acc: %.3f ' %(step, g_loss,
                                                d_loss1, d_loss2,
                                                c_loss, c_acc*100,
                                                 acc_train*100, acc_test*100))#, end = '\r')
        f_history.write(f"{step},{g_loss},{d_loss1},{d_loss2},{c_loss},{c_acc*100},{loss_train},{loss_test},{acc_train*100},{acc_test*100}\n")
        
        if step==1:
            plot_data(unnormalize(X_test).astype(int), 0, "test", grid_size = [10, 10], OUT_PATH=LOG_PATH)
        # evaluate the model performance every so often
        if (step) % (100) == 0 or step == 1:
            #summarize_performance(step, generator_model, supervised_model, latent_dim, dataset, dataset_test)
            # prepare fake examples
            X_generated, _ = generate_fake_samples(generator_model, latent_dim, n_samples=100)
            # scale from [-1,1] to [0,255]
            plot_data(unnormalize(X_generated).astype(int), step, "generated", grid_size = [10, 10], OUT_PATH=LOG_PATH)
            
            X_train, y_train = dataset
            _, acc = supervised_model.evaluate(X_train, y_train, verbose=0)
            print('Train Classifier Accuracy: %.3f%%\n' % (acc * 100))
            
            # evaluate the test classifier model
            X_test, y_test = dataset_test
            _, acc = supervised_model.evaluate(X_test, y_test, verbose=0)
            print('Test Classifier Accuracy: %.3f%%\n' % (acc * 100))
            # save the generator model
            filename2 = f'{LOG_PATH}generator_model_{step}.h5'
            generator_model.save(filename2)
            # save the classifier model
            filename3 = f'{LOG_PATH}supervised_model_{step}.h5'
            supervised_model.save(filename3)
            print('>Saving models Generator: %s and Supervised: %s' % (filename2, filename3))
    
    f_history.close()

In [ ]:
# create the discriminator models
unsupervised_model, supervised_model = define_discriminator(in_shape=input_shape,n_classes=num_classes)
# create the generator
generator_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(generator_model, unsupervised_model)
# load image data
dataset, dataset_test = load_real_samples()
# train model
train(generator_model, unsupervised_model, supervised_model, gan_model, dataset, dataset_test, latent_dim, n_epochs=epochs, n_batch=batch_size, percent_samples=labeled_rate)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Adrian\anaconda3\envs\tf15\lib\site-packages\IPython\core\interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-22-2e42f09cc497>", line 2, in <module>
    unsupervised_model, supervised_model = define_discriminator(in_shape=input_shape,n_classes=num_classes)
  File "<ipython-input-13-2bb972a89b0f>", line 7, in define_discriminator
    fe = WeightNormalization(Conv2D(96, (3,3), strides=(1,1), padding='same'), data_init=False)(fe)
NameError: name 'WeightNormalization' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Adrian\anaconda3\envs\tf15\lib\site-packages\IPython\core\interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another excep

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Adrian\anaconda3\envs\tf15\lib\site-packages\IPython\core\interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-22-2e42f09cc497>", line 2, in <module>
    unsupervised_model, supervised_model = define_discriminator(in_shape=input_shape,n_classes=num_classes)
  File "<ipython-input-13-2bb972a89b0f>", line 7, in define_discriminator
    fe = WeightNormalization(Conv2D(96, (3,3), strides=(1,1), padding='same'), data_init=False)(fe)
NameError: name 'WeightNormalization' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Adrian\anaconda3\envs\tf15\lib\site-packages\IPython\core\interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another excep

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Adrian\anaconda3\envs\tf15\lib\site-packages\IPython\core\interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-22-2e42f09cc497>", line 2, in <module>
    unsupervised_model, supervised_model = define_discriminator(in_shape=input_shape,n_classes=num_classes)
  File "<ipython-input-13-2bb972a89b0f>", line 7, in define_discriminator
    fe = WeightNormalization(Conv2D(96, (3,3), strides=(1,1), padding='same'), data_init=False)(fe)
NameError: name 'WeightNormalization' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Adrian\anaconda3\envs\tf15\lib\site-packages\IPython\core\interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another excep

### Testing

In [22]:
dataset, dataset_test = load_real_samples()

ERROR! Session/line number was not unique in database. History logging moved to new session 4632
170500096/170498071 [==============================] - 126s 1us/step
(50000, 32, 32, 3) (50000,) (10000, 32, 32, 3) (10000,)


In [ ]:
X_train, y_train = dataset
_, acc = supervised_model.evaluate(X_train, y_train, verbose=0)
print('Train Classifier Accuracy: %.3f%%\n' % (acc * 100))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Adrian\anaconda3\envs\tf15\lib\site-packages\IPython\core\interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-23-fc2742eda586>", line 2, in <module>
    _, acc = supervised_model.evaluate(X_train, y_train, verbose=0)
NameError: name 'supervised_model' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Adrian\anaconda3\envs\tf15\lib\site-packages\IPython\core\interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Adrian\anaconda3\envs\tf15\lib\site-packages\IPython\core\ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_line

In [ ]:
X_test, y_test = dataset_test
_, acc = supervised_model.evaluate(X_test, y_test, verbose=0)
print('Test Classifier Accuracy: %.3f%%\n' % (acc * 100))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Adrian\anaconda3\envs\tf15\lib\site-packages\IPython\core\interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-23-594587932d97>", line 2, in <module>
    _, acc = supervised_model.evaluate(X_test, y_test, verbose=0)
NameError: name 'supervised_model' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Adrian\anaconda3\envs\tf15\lib\site-packages\IPython\core\interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Adrian\anaconda3\envs\tf15\lib\site-packages\IPython\core\ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_

### Plotting

In [23]:
import pandas as pd

ERROR! Session/line number was not unique in database. History logging moved to new session 4634


In [ ]:
results_file = pd.read_csv(f"{LOG_PATH}SSL_GAN.csv")

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Adrian\anaconda3\envs\tf15\lib\site-packages\IPython\core\interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-24-3876b58a229b>", line 1, in <module>
    results_file = pd.read_csv(f"{LOG_PATH}SSL_GAN.csv")
  File "C:\Users\Adrian\anaconda3\envs\tf15\lib\site-packages\pandas\io\parsers.py", line 610, in read_csv
    return _read(filepath_or_buffer, kwds)
  File "C:\Users\Adrian\anaconda3\envs\tf15\lib\site-packages\pandas\io\parsers.py", line 462, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
  File "C:\Users\Adrian\anaconda3\envs\tf15\lib\site-packages\pandas\io\parsers.py", line 819, in __init__
    self._engine = self._make_engine(self.engine)
  File "C:\Users\Adrian\anaconda3\envs\tf15\lib\site-packages\pandas\io\parsers.py", line 1050, in _make_engine
    return mapping[engine](self.f, **self.options)  # type: ignore[call-arg]
  File "C:\Users

In [ ]:
log_file = results_file.iloc[:,1:]
log_file

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Adrian\anaconda3\envs\tf15\lib\site-packages\IPython\core\interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-24-abd2a957be53>", line 1, in <module>
    log_file = results_file.iloc[:,1:]
NameError: name 'results_file' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Adrian\anaconda3\envs\tf15\lib\site-packages\IPython\core\interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Adrian\anaconda3\envs\tf15\lib\site-packages\IPython\core\ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "

In [ ]:
log_file.iloc[:, [0,1,2,3]].plot(figsize=(12,8))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Adrian\anaconda3\envs\tf15\lib\site-packages\IPython\core\interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-24-1553fc5b942b>", line 1, in <module>
    log_file.iloc[:, [0,1,2,3]].plot(figsize=(12,8))
NameError: name 'log_file' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Adrian\anaconda3\envs\tf15\lib\site-packages\IPython\core\interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Adrian\anaconda3\envs\tf15\lib\site-packages\IPython\core\ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset

In [ ]:
log_file.iloc[:, [5,6]].plot(figsize=(12,8))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Adrian\anaconda3\envs\tf15\lib\site-packages\IPython\core\interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-24-0ac82bf17dda>", line 1, in <module>
    log_file.iloc[:, [5,6]].plot(figsize=(12,8))
NameError: name 'log_file' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Adrian\anaconda3\envs\tf15\lib\site-packages\IPython\core\interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Adrian\anaconda3\envs\tf15\lib\site-packages\IPython\core\ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  

In [ ]:
log_file.iloc[:, [7,8]].plot(figsize=(12,8), ylim=(0,100), yticks=range(0,110,10))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Adrian\anaconda3\envs\tf15\lib\site-packages\IPython\core\interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-24-72b8f7dbbb06>", line 1, in <module>
    log_file.iloc[:, [7,8]].plot(figsize=(12,8), ylim=(0,100), yticks=range(0,110,10))
NameError: name 'log_file' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Adrian\anaconda3\envs\tf15\lib\site-packages\IPython\core\interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Adrian\anaconda3\envs\tf15\lib\site-packages\IPython\core\ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, num